In [67]:
import soccerdata
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException

In [68]:
base_url = "https://sofifa.com/players?type=all&lg%5B0%5D=13&lg%5B1%5D=31&lg%5B2%5D=19&lg%5B3%5D=53&lg%5B4%5D=16&showCol%5B0%5D=ae&showCol%5B1%5D=oa&showCol%5B2%5D=pt&showCol%5B3%5D=vl&showCol%5B4%5D=wg&showCol%5B5%5D=tt&showCol%5B6%5D=pi&showCol%5B7%5D=wi&showCol%5B8%5D=pf&showCol%5B9%5D=bo&showCol%5B10%5D=hi&showCol%5B11%5D=bp&showCol%5B12%5D=jt&showCol%5B13%5D=le&showCol%5B14%5D=gu&showCol%5B15%5D=rc&showCol%5B16%5D=cp&showCol%5B17%5D=at&showCol%5B18%5D=ps2&showCol%5B19%5D=ps1&showCol%5B20%5D=t2&showCol%5B21%5D=t1&showCol%5B22%5D=phy&showCol%5B23%5D=def&showCol%5B24%5D=pas&showCol%5B25%5D=pac&showCol%5B26%5D=dri&showCol%5B27%5D=hc&showCol%5B28%5D=bt&showCol%5B29%5D=ir&showCol%5B30%5D=aw&showCol%5B31%5D=sk&showCol%5B32%5D=dw&showCol%5B33%5D=bs&showCol%5B34%5D=sho&showCol%5B35%5D=gd&showCol%5B36%5D=sa&showCol%5B37%5D=td&showCol%5B38%5D=cj&showCol%5B39%5D=wk&showCol%5B40%5D=tc&showCol%5B41%5D=ma&showCol%5B42%5D=cm&showCol%5B43%5D=vi&showCol%5B44%5D=pe&showCol%5B45%5D=in&showCol%5B46%5D=ar&showCol%5B47%5D=po&showCol%5B48%5D=te&showCol%5B49%5D=st&showCol%5B50%5D=ju&showCol%5B51%5D=so&showCol%5B52%5D=tp&showCol%5B53%5D=sr&showCol%5B54%5D=ln&showCol%5B55%5D=ag&showCol%5B56%5D=ba&showCol%5B57%5D=re&showCol%5B58%5D=ac&showCol%5B59%5D=to&showCol%5B60%5D=sp&showCol%5B61%5D=lo&showCol%5B62%5D=cu&showCol%5B63%5D=dr&showCol%5B64%5D=fr&showCol%5B65%5D=ts&showCol%5B66%5D=he&showCol%5B67%5D=fi&showCol%5B68%5D=ta&showCol%5B69%5D=sh&showCol%5B70%5D=cr&showCol%5B71%5D=vo&showCol%5B72%5D=bl&count=100&r=240050&offset=60"

In [69]:
seasons = {
    2025: "250044",
    2024: "240050",
    2023: "230054",
    2022: "220069",
    2021: "210064",
    2020: "200061",
    2019: "190075",
    2018: "180084",
    2017: "170099",
    2016: "160058",
    2015: "150059",
    2014: "140052", 
    2013: "130034",
    2012: "120002",
    2011: "110002",
    2010: "100002",
    2009: "090002",
    2008: "080002",
    2007: "070002"
}

In [51]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)


In [ ]:
from pathlib import Path

chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

# Add user agent
chrome_options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

# Performance optimizations
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--disable-extensions')


driver = webdriver.Chrome(options=chrome_options)
print("Chrome WebDriver initialized")

# Define save path
SAVE_DIR = Path("../data/raw/raw_sofifa/yearly")

def get_team_stats(driver, season_code, year):
    """
    Scrape team stats for a given season and offset.
    Driver should be passed in to reuse across multiple calls.
    """
    from selenium.common.exceptions import TimeoutException
    
    df = pd.DataFrame()
    offset = 0
    while True:
        try:
            driver.get(f"{base_url}&r={season_code}&offset={offset}")
            print(f"Season C: {season_code}, Offset: {offset} - Page loaded")
        
            time.sleep(3)
            
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.TAG_NAME, "table"))
            )
            
        except TimeoutException:
            print(f"No more data found at offset {offset}. Scraping complete!")
            break
        except Exception as e:
            print(f"Error occurred: {e}")
            break
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        table = soup.find('table')  # Find first table
        
        if not table:
            print("ERROR: No table found!")
            break
        
        headers = [th.get_text(strip=True) for th in table.select("thead th")]
        
        data = []
        for row in table.select("tbody tr"):
            cols = row.find_all(['th', 'td'])
            if cols:
                data.append([col.get_text(strip=True) for col in cols])

        temp_df = pd.DataFrame(data, columns=headers)

        offset += 60
        df = pd.concat([df, temp_df], ignore_index=True)
        
        if offset % 600 == 0:
            print(f"{offset} number of players collected so far...")
        
    df['season'] = year
    df['season_code'] = season_code

    print(f"\n{'='*60}")
    print(f"Total rows collected: {len(df)}")
    print(f"{'='*60}")
    return df

for year, season_code in seasons.items():
    print(f"\n{'='*60}")
    print(f"Starting scrape for year {year} (season code: {season_code})")
    print(f"{'='*60}")
    
    df = get_team_stats(driver, season_code, year)
    output_path = SAVE_DIR / f"sofifa_players_{year}.csv"
    df.to_csv(output_path, index=False)
    print(f"{year} Data Saved to: {output_path}")


print(f"\nScraping complete! DataFrame shape: {df.shape}")

In [ ]:

"""
# def collect_all(self):
    #     """Combine all yearly CSV files into one master file."""
    #     df_list = []
        
    #     for year in self.seasons.keys():
    #         file_path = self.save_dir / f"sofifa_players_{year}.csv"
    #         if file_path.exists():
    #             yearly_df = pd.read_csv(file_path)
    #             df_list.append(yearly_df)
    #         else:
    #             print(f"Warning: {file_path} not found, skipping...")
        
    #     # Combine all DataFrames at once
    #     df = pd.concat(df_list, ignore_index=True)
        
    #     output_path = self.save_dir.parent / "sofifa_players_all_years.csv"
    #     df.to_csv(output_path, index=False)
        
    #     print(f"All years combined ({len(df)} rows) and saved to: {output_path}")


    # def yearly_scrape(self):
    #     """Scrape all seasons and save individual year files."""
    #     # Ensure directory exists
    #     self.save_dir.mkdir(parents=True, exist_ok=True)
        
    #     driver = self.initialize_driver()
        
    #     try:
    #         for year, season_code in self.seasons.items():
    #             print(f"\n{'='*60}")
    #             print(f"Starting year {year} (season code: {season_code})")
    #             print(f"{'='*60}")
                
    #             df = self.get_individual_stats(driver, season_code, year)
    #             output_path = self.save_dir / f"sofifa_players_{year}.csv"
    #             df.to_csv(output_path, index=False)
    #             print(f"✓ {year} data saved to: {output_path}\n")
            
        
    #     finally:
    #         # Always close driver, even if error occurs
    #         self.close_driver(driver)
"""